# Does the local gateway help the fullnode to receive pending transactions faster?

This notebook compares pending transactions receiving speed between with and without gateways.

Steps:

* First, start a fullnode with a local bloXroute gateway, and run the [bloxroute_tx](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/bloxroute/tx/bloxroute_tx.go) and [fullnode_tx](https://github.com/crypto-crawler/fullnode-benchmarks/blob/main/cmd/fullnode/tx/fullnode_tx.go) to collect data
* Second, stop the local gateway, and run `bloxroute_tx` and `fullnode_tx` to collect data again
* Substract the `received_at` timestamp of txs with the same hash, we get to know the gap between with and without gateway.

Note: Make sure the BSC fullnode is fully synced before this benchmark, type `eth.syncing` in geth console and make sure it is `faluse`, which means the fullnode is synced.

Hardware:

* A machine with 128 cores, 256GB memory in a Fremont IDC
* A `m5zn.3xlarge` instance in AWS Virginia regon

Files:

* `bloxroute-newtxs-cloud-fremont-with-gateway.json.gz`, `bloxroute-newtxs-cloud-virginia-with-gateway.json.gz`, `fullnode-tx-fremont-with-gateway.json.gz` and `fullnode-tx-virginia-with-gateway.json.gz` are generated in step 1
* `bloxroute-newtxs-cloud-fremont-without-gateway.json.gz`, `bloxroute-newtxs-cloud-virginia-without-gateway.json.gz`, `fullnode-tx-fremont-without-gateway.json.gz` and `fullnode-tx-virginia-without-gateway.json.gz` are generated in step 2

In [2]:
import gzip
import json
import pandas as pd
from typing import Dict

In [3]:
pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

In [4]:
def read_timestamp(file: str) -> Dict[str, int]:
    result: Dict[str, int] = {}  # hash -> received_at
    f_in = gzip.open(file, "rt") if file.endswith('.json.gz') else open(file, "rt")
    with f_in:
        for line in f_in:
            obj = json.loads(line)
            if 'received_at' in obj:
                if 'hash' in obj:  # fullnode
                    result[obj['hash']] = obj['received_at']
                elif 'txHash' in obj:  # bloXroute
                    result[obj['txHash']] = obj['received_at']
                else:
                    raise ValueError(line)
    return result

In [5]:
def compare_timestamp(file1: str, file2: str) -> pd.DataFrame:
    dict1 = read_timestamp(file1)
    dict2 = read_timestamp(file2)
    diff = {}
    for tx_hash, received_at in dict1.items():
        if tx_hash in dict2:
            diff[tx_hash] = received_at-dict2[tx_hash]
    
    df = pd.DataFrame(diff.items(), columns=['tx_hash', 'gap'])
    # remove outliers
    quantile05 = df['gap'].quantile(0.05)
    quantile95 = df['gap'].quantile(0.95)
    return df[(df['gap'] >= quantile05) & (df['gap']<=quantile95)]

## In Fremont IDC

In [10]:
compare_timestamp('./data/bloxroute-newtxs-cloud-fremont-without-gateway.json.gz', './data/fullnode-tx-fremont-without-gateway.json.gz').describe()

,gap
count,144345.000000
mean,-133.604046
std,186.355508
min,-621.000000
25%,-111.000000
50%,-65.000000
75%,-13.000000
max,24.000000


In [11]:
compare_timestamp('./data/bloxroute-newtxs-cloud-fremont-with-gateway.json.gz', './data/fullnode-tx-fremont-with-gateway.json.gz').describe()

,gap
count,142864.000000
mean,-116.179996
std,174.895064
min,-612.000000
25%,-98.000000
50%,-52.000000
75%,-6.000000
max,26.000000


**Conclusion: The cloud API is 65ms faster than the fullnode without gateway, and 52ms faster than the fullnode with a gateway, which means the fullnode is 13ms faster after the local gateway is running.**

## In AWS Virginia

In [12]:
compare_timestamp('./data/bloxroute-newtxs-cloud-virginia-without-gateway.json.gz', './data/fullnode-tx-virginia-without-gateway.json.gz').describe()

,gap
count,144477.000000
mean,-227.121895
std,214.747226
min,-609.000000
25%,-451.000000
50%,-144.000000
75%,-19.000000
max,32.000000


In [13]:
compare_timestamp('./data/bloxroute-newtxs-cloud-virginia-with-gateway.json.gz', './data/fullnode-tx-virginia-with-gateway.json.gz').describe()

,gap
count,143942.000000
mean,-214.868211
std,213.300921
min,-599.000000
25%,-443.000000
50%,-116.000000
75%,-10.000000
max,17.000000


**Conclusion: Without a local gateway the gap between the fullnode and bloXroute cloud is 144ms, while with a local gateway the gap reduces to 116ms, so the local gateway obviously helps the local fullnode receive transactions faster!**

**Conclusion: The cloud API is 144ms faster than the fullnode without gateway, and 116ms faster than the fullnode with a gateway, which means the fullnode is 28ms faster after the local gateway is running.**

## Compare two gateways

In [16]:
compare_timestamp('./data/bloxroute-newtxs-gateway-virginia.json.gz', './data/bloxroute-newtxs-gateway-fremont.json.gz').describe()

,gap
count,150142.000000
mean,-6.567643
std,8.363417
min,-16.000000
25%,-12.000000
50%,-10.000000
75%,-4.000000
max,15.000000


**Conclusion: The gateway in AWS Virginia is 10ms faster than the one in Fremont, because there are some bloXroute relay nodes in AWS, so the gateway in AWS Virginia is closer to relay nodes.**

**Overall, this benchmark shows that the local gateway DOES help the fullnode receive pending transactions faster.**

Gaps: Virginia gateway -> 1ms -> Virginia Cloud -> 10ms -> Fremont gateway -> 0 -> Fremont Cloud -> 52ms -> Fremont fullnode with gateway -> 12ms -> Virginia fullnode with gateway